In [ ]:
!pip install requests

In [ ]:
import requests
import json
import time
import math

import numpy as np
import pandas as pd
# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from google.colab import files
from datetime import date, datetime

In [ ]:
def addColumnPrefix(cols, prefix):
  columns_rename_dict={}
  for col in cols:
    columns_rename_dict[col]=prefix+'-'+col
  return columns_rename_dict

## Data Collection

In [ ]:
start_date="2022-09-01"
# start_date="2023-01-01"
end_date="2023-03-01"

In [ ]:
# Uses limit = current_date - start_date + 1
current_date_obj=date.today()
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d').date()
limit=(current_date_obj-start_date_obj).days + 2

### NFT Market Sales Statistics

In [ ]:
stats={}
url="https://nonfungible.com/api/statistics?metrics[]=count-sale&metrics[]=sum-usd&metrics[]=avg-usd&metrics[]=count-salesprimary&metrics[]=sum-usdprimary&metrics[]=count-salessecondary&metrics[]=sum-usdsecondary&metrics[]=unique-wallets&metrics[]=unique-buyer&metrics[]=unique-seller&avg=1&limit="+str(limit)+"&comparison=false&includeLatest=true&useCache=false"
response = requests.get(url)
print(url)
data=json.loads(response.text)['data']
parameters=['count-sale','sum-usd','avg-usd','count-salesprimary','sum-usdprimary','count-salessecondary','sum-usdsecondary','unique-wallets','unique-buyer','unique-seller']
for parameter in parameters:
  results=data[parameter]
  for result in results:
    if result[0] not in stats.keys():
      stats[result[0]]={}
    stats[result[0]][parameter]=result[1]

In [ ]:
market_sales_stats = pd.DataFrame.from_dict(stats,orient='index').reset_index()

In [ ]:
market_sales_stats['index']=market_sales_stats['index']+86400000

In [ ]:
market_sales_stats.rename(columns = {'index':'timestamp'}, inplace = True)
renamed_columns=addColumnPrefix(['count-sale', 'sum-usd', 'avg-usd', 'count-salesprimary',
       'sum-usdprimary', 'count-salessecondary', 'sum-usdsecondary',
       'unique-wallets', 'unique-buyer', 'unique-seller'],'market')
market_sales_stats.rename(columns = renamed_columns, inplace = True)

In [ ]:
market_sales_stats.isnull().sum()

In [ ]:
market_sales_stats=market_sales_stats.dropna()

In [ ]:
market_sales_stats=market_sales_stats[market_sales_stats['market-count-sale']!=0]
market_sales_stats=market_sales_stats[market_sales_stats['market-sum-usd']!=0]

In [ ]:
market_sales_stats

In [ ]:
market_sales_stats.to_csv('market_sales_stats.csv',index=False)

### Collection Categories Sales Statistics 

In [ ]:
categories=['collectible','art','metaverse','game']

In [ ]:
category_sales_stats={}
for category in categories:
  stats={}
  url="https://nonfungible.com/api/statistics?metrics[]=count-sale&metrics[]=sum-usd&metrics[]=avg-usd&metrics[]=count-salesprimary&metrics[]=sum-usdprimary&metrics[]=count-salessecondary&metrics[]=sum-usdsecondary&metrics[]=unique-wallets&metrics[]=unique-buyer&metrics[]=unique-seller&avg=1&limit="+str(limit)+"&comparison=false&category="+category+"&includeLatest=true&useCache=false"
  response = requests.get(url)
  data=json.loads(response.text)['data']
  parameters=['count-sale','sum-usd','avg-usd','count-salesprimary','sum-usdprimary','count-salessecondary','sum-usdsecondary','unique-wallets','unique-buyer','unique-seller']
  for parameter in parameters:
    results=data[parameter]
    for result in results:
      # print(result)
      if result[0] not in stats.keys():
        stats[result[0]]={}
      stats[result[0]][parameter]=result[1]
      
  category_sales_stats[category] = pd.DataFrame.from_dict(stats,orient='index').reset_index()
  category_sales_stats[category]['index']=category_sales_stats[category]['index']+86400000
  category_sales_stats[category].rename(columns = {'index':'timestamp'}, inplace = True)
  renamed_columns=addColumnPrefix(['count-sale', 'sum-usd', 'avg-usd', 'count-salesprimary','sum-usdprimary', 'count-salessecondary', 'sum-usdsecondary','unique-wallets', 'unique-buyer', 'unique-seller'],category)
  category_sales_stats[category].rename(columns = renamed_columns , inplace = True)
  print('Category:',category)
  print(category_sales_stats[category])
  category_sales_stats[category].to_csv(category+'_sales_stats.csv',index=False)

### NFT Collections Metadata

In [ ]:
# # If I extract all collections of a project
# collections={}

# url='https://nonfungible.com/api/topProjects?limit=1000&orderBy=SUMUSD7D&order=DESC'
# response = requests.get(url)
# projects=json.loads(response.text)['projects']
# for project in projects:
#   collection={

#       'project':project['title'] if 'title' in project.keys() else None,

#       'projectSlug':project['slug'] if 'slug' in project.keys() else None,
      
#       'date':project['date'] if 'date' in project.keys() else None,

#       'description':project['custom_fields_project']['description'] 
#       if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
#       and 'description' in project['custom_fields_project'].keys() 
#       else None,

#       'discord':project['custom_fields_project']['socialMedia']['discord'] 
#       if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
#       and 'socialMedia' in project['custom_fields_project'].keys() and project['custom_fields_project']['socialMedia']!=None
#       and 'discord' in project['custom_fields_project']['socialMedia'].keys() 
#       else None,

#       'twitter':project['custom_fields_project']['socialMedia']['twitter'] 
#       if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
#       and 'socialMedia' in project['custom_fields_project'].keys() and project['custom_fields_project']['socialMedia']!=None
#       and 'twitter' in project['custom_fields_project']['socialMedia'].keys() 
#       else None
#   }
#   if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None:
#     for i in range(len(project['custom_fields_project']['smartContracts'])):
#         collection['title']=project['custom_fields_project']['smartContracts'][i]['label'] if 'label' in project['custom_fields_project']['smartContracts'][i].keys() else project['title'] if 'title' in project.keys() else None
#         collection['category']=project['custom_fields_project']['smartContracts'][i]['category'] if 'category' in project['custom_fields_project']['smartContracts'][i].keys() else None
#         collection['address']=project['custom_fields_project']['smartContracts'][i]['address'] if 'address' in project['custom_fields_project']['smartContracts'][i].keys() else None
#         collection['token']=project['custom_fields_project']['smartContracts'][i]['tokenTickerSymbol'] if 'tokenTickerSymbol' in project['custom_fields_project']['smartContracts'][i].keys() else None
#         collection['type']=project['custom_fields_project']['smartContracts'][i]['type'][0] if 'type' in project['custom_fields_project']['smartContracts'][i].keys() else None
#         collection['slug']=collection['title'].replace(" ", "").lower()
#         print(collection)
#         # if collection['address']==None:
#             # continue
#         collections[collection['slug']]=collection.copy()


In [ ]:
collections={}

url='https://nonfungible.com/api/topProjects?limit=1000&orderBy=SUMUSD7D&order=DESC'
response = requests.get(url)
projects=json.loads(response.text)['projects']
for project in projects:
  collection={
      'title':project['custom_fields_project']['smartContracts'][0]['label']  
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None
      and len(project['custom_fields_project']['smartContracts'])>0 
      and 'label' in project['custom_fields_project']['smartContracts'][0].keys() 
      else project['title'] if 'title' in project.keys() else None,

      'project':project['title'] if 'title' in project.keys() else None,

      'projectSlug':project['slug'] if 'slug' in project.keys() else None,
      
      'date':project['date'] if 'date' in project.keys() else None,

      'description':project['custom_fields_project']['description'] 
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'description' in project['custom_fields_project'].keys() 
      else None,

      'address':project['custom_fields_project']['smartContracts'][0]['address']  
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None
      and len(project['custom_fields_project']['smartContracts'])>0 
      and 'address' in project['custom_fields_project']['smartContracts'][0].keys() 
      else None,

      # 'category':project['custom_fields_project']['smartContracts'][0]['category'] 
      # if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      # and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None
      # and len(project['custom_fields_project']['smartContracts'])>0 
      # and 'category' in project['custom_fields_project']['smartContracts'][0].keys() 
      # else None,

      'category': project['projectCategories'][0]['slug'] 
      if 'projectCategories' in project.keys() and project['projectCategories']!=None
      and len(project['projectCategories'])>0 and 'slug' in project['projectCategories'][0].keys() 
      else None,

      'token':project['custom_fields_project']['smartContracts'][0]['tokenTickerSymbol'] 
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None
      and len(project['custom_fields_project']['smartContracts'])>0 
      and 'tokenTickerSymbol' in project['custom_fields_project']['smartContracts'][0].keys() 
      else None,

      'type':project['custom_fields_project']['smartContracts'][0]['type'][0] 
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'smartContracts' in project['custom_fields_project'].keys() and project['custom_fields_project']['smartContracts']!=None
      and len(project['custom_fields_project']['smartContracts'])>0 
      and 'type' in project['custom_fields_project']['smartContracts'][0].keys() 
      else None,

      'discord':project['custom_fields_project']['socialMedia']['discord'] 
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'socialMedia' in project['custom_fields_project'].keys() and project['custom_fields_project']['socialMedia']!=None
      and 'discord' in project['custom_fields_project']['socialMedia'].keys() 
      else None,

      'twitter':project['custom_fields_project']['socialMedia']['twitter'] 
      if 'custom_fields_project' in project.keys() and project['custom_fields_project']!=None
      and 'socialMedia' in project['custom_fields_project'].keys() and project['custom_fields_project']['socialMedia']!=None
      and 'twitter' in project['custom_fields_project']['socialMedia'].keys() 
      else None
  }
  collection['slug']=collection['title'].replace(" ", "").lower()
  if collection['address']==None:
    continue
  collections[collection['slug']]=collection


In [ ]:
collections

In [ ]:
print('Number of collections: ',len(collections.keys()))

Number of collections:  1100


In [ ]:
for collectionSlug in collections.keys():
  url = "https://eth-mainnet.g.alchemy.com/nft/v2/p4bGU_z8jpNfDOf-kMNuQg5nCzkYQw4u/getContractMetadata?contractAddress="+collections[collectionSlug]['address']
  response = requests.get(url)
  if response.status_code == 200:
    result=response.json()
    collections[collectionSlug]['token']=result['contractMetadata']['symbol'] if collections[collectionSlug]['token']== None and 'symbol' in result['contractMetadata'].keys() else collections[collectionSlug]['token']
    # collections[collectionSlug]['totalSupply']=int(result['contractMetadata']['totalSupply']) if 'totalSupply' in result['contractMetadata'].keys() else None
    collections[collectionSlug]['description']=result['contractMetadata']['openSea']['description'] if collections[collectionSlug]['description']== None and 'openSea' in result['contractMetadata'].keys() and 'description' in result['contractMetadata']['openSea'].keys() else collections[collectionSlug]['description']
    collections[collectionSlug]['twitter']=result['contractMetadata']['openSea']['twitterUsername'] if collections[collectionSlug]['twitter']== None and 'openSea' in result['contractMetadata'].keys() and 'twitterUsername' in result['contractMetadata']['openSea'].keys() else collections[collectionSlug]['twitter']
    collections[collectionSlug]['discord']=result['contractMetadata']['openSea']['discordUrl'] if collections[collectionSlug]['discord']== None and 'openSea' in result['contractMetadata'].keys() and 'discordUrl' in result['contractMetadata']['openSea'].keys() else collections[collectionSlug]['discord']
  # else:
    # collections[collectionSlug]['totalSupply']=None
    
  # For adding traits summary as part of collection data
  # traits_url = "https://eth-mainnet.g.alchemy.com/nft/v2/p4bGU_z8jpNfDOf-kMNuQg5nCzkYQw4u/summarizeNFTAttributes?contractAddress="+collections[collectionSlug]['address']
  # traits_response = requests.get(traits_url)
  # print(traits_response.text)
  # if traits_response.status_code == 200:
  #   traits_result=traits_response.json()
  #   collections[collectionSlug]['totalTraits']=len(result['summary']) if 'summary' in result.keys() else None
  # else:
  #   collections[collectionSlug]['totalTraits']=None


In [ ]:
collection_df = pd.DataFrame.from_dict(collections,orient='index')

In [ ]:
collection_df.isnull().sum()

In [ ]:
collection_df=collection_df.dropna()

In [ ]:
collection_df

In [ ]:
collection_df.to_csv('collections.csv')

#### Dividing into categories

In [ ]:
collection_df.category.unique()

In [ ]:
collectible_df=collection_df.query("category=='collectible'")
collectibles=collectible_df.to_dict('records')

In [ ]:
collectible_df

### NFT Project Sales Statistics

In [ ]:
for collection in collectibles:
    print(collection['project'],' ',collection['title'],' ', collection['projectSlug'] )

In [ ]:
project_sales_stats=pd.DataFrame()

for collection in collectibles:
  # Gets sale statistics of entire project
  url="https://web-api.cryptoslam.io/web-api/v1/sales/"+collection['project'].replace('(','').replace(')','').replace(' ','-').lower()+"/summary?groupingtype=daily"
  response = requests.get(url)
  if response.status_code ==200:
    data=json.loads(response.text)['salesSummaryRecords']
    df = pd.DataFrame.from_dict(data)
    df['date'] = df['date'].apply(lambda x: int(datetime.timestamp(datetime.strptime(x.split('T')[0],"%Y-%m-%d")))*1000)
    df['project']=collection['slug']
    df['index']=df['date']
    df['index']=df['index'].apply(lambda x: collection['slug'] + str(x))
    if project_sales_stats.empty:
      project_sales_stats=df
    else: 
      project_sales_stats=pd.concat([project_sales_stats,df])
  else:
    print('QUERY:',url)
    print('RESPONSE:',response.text)

In [ ]:
project_sales_stats=project_sales_stats.set_index('index')

In [ ]:
project_sales_stats.rename(columns = {'date':'timestamp'}, inplace = True)
renamed_columns=addColumnPrefix(['salesUSD', 'salesToken', 'uniqueBuyers', 'sellers',
       'totalTransactions', 'activeOwners', 'totalOwners', 'transfers',
       'mints', 'burns', 'avgPriceUSD', 'avgPriceToken'],'project')
project_sales_stats.rename(columns = renamed_columns , inplace = True)

In [ ]:
project_sales_stats['timestamp']=project_sales_stats['timestamp']+86400000

In [ ]:
project_sales_stats=project_sales_stats[project_sales_stats['project-salesUSD']!=0]
project_sales_stats=project_sales_stats[project_sales_stats['project-salesToken']!=0]
project_sales_stats=project_sales_stats[project_sales_stats['project-avgPriceUSD']!=0]
project_sales_stats=project_sales_stats[project_sales_stats['project-avgPriceToken']!=0]

In [ ]:
project_sales_stats['tokenUSD']= project_sales_stats['project-salesUSD']/project_sales_stats['project-salesToken']

In [ ]:
project_sales_stats.isnull().sum()

In [ ]:
project_sales_stats=project_sales_stats.dropna()

In [ ]:
project_sales_stats

In [ ]:
project_sales_stats.project.unique()

In [ ]:
project_sales_stats.to_csv('project_sales_stats.csv',index=False)

In [ ]:
# If using nonfungible.com to get project sales statistics

# stats={}
# for collection in collectibles[:10]:
#   # Gets sale statistics of entire project
#   url="https://nonfungible.com/api/statistics?metrics[]=count-sale&metrics[]=sum-usd&metrics[]=avg-usd&metrics[]=count-salesprimary&metrics[]=sum-usdprimary&metrics[]=count-salessecondary&metrics[]=sum-usdsecondary&metrics[]=unique-wallets&metrics[]=unique-buyer&metrics[]=unique-seller&avg=1&limit="+str(limit)+"&comparison=false&project="+collection['projectSlug']+"&includeLatest=true&useCache=false"
#   response = requests.get(url)
#   data=json.loads(response.text)['data']
#   parameters=['count-sale','sum-usd','avg-usd','count-salesprimary','sum-usdprimary','count-salessecondary','sum-usdsecondary','unique-wallets','unique-buyer','unique-seller']
#   for parameter in parameters:
#     results=data[parameter]
#     for result in results:
#       # print(result)
#       if collection['slug']+str(result[0]) not in stats.keys():
#         stats[collection['slug']+str(result[0])]={
#             'timestamp':result[0],
#             'project':collection['slug']
#         }
#       stats[collection['slug']+str(result[0])][parameter]=result[1]
    

#### Google Trends

In [ ]:
# hl="en-GB"
# tz="-330"
# resolution="DAY"
# locale="en-GB"
# # List of token keys corresponding to their search value
# tokens={
#     'boredapeyachtclub':'APP6_UEAAAAAZBXWrk-NhPND-sutw8nNjhMXSImpDmQz',
#     'cryptopunks':'APP6_UEAAAAAZBXY1q3ImoMi_cuz0bdiMegphoct3c2h',
#     'azuki':'APP6_UEAAAAAZBXY65ouQyqwmY3qPCpa8V4KQIZ9LLrz',
#     'moonbirds':'APP6_UEAAAAAZBXZAolULwbQDP_zDHoo6wQV-P3o538g',
#     'doodles':'APP6_UEAAAAAZBXZFWC118PseG6wH0k05OxATH_dTx5z',
#     'clonex':'APP6_UEAAAAAZBXZMHxnAbhPl2ZQJQGuBzrECv6D0Y8c',
#     'pudgypenguins':'APP6_UEAAAAAZBXZRg9qYFxFsJoXXZnKKxWQhNf4LbLw',
#     'potatoz':'APP6_UEAAAAAZBXZbutHqrrvHNrL5-0yhp6YJEIay-kR',
#     'onchainmonkey':'APP6_UEAAAAAZBXfDDv2gWGnYCFR2yTx8p6sqpt9mnr5',
#     '0n1force':'APP6_UEAAAAAZBXf34Ektvy_6stkhcZCnI1s_Lk8Byrz'
# }

In [ ]:
# for collection in collectibles[:10]:
#   data=""
#   search="https://trends.google.com/trends/api/widgetdata/multiline?hl="+hl+"&tz="+tz+"&req=%7B%22time%22:%22"+start_date+"+"+end_date+"%22,%22resolution%22:%22"+resolution+"%22,%22locale%22:%22"+locale+"%22,%22comparisonItem%22:%5B%7B%22geo%22:%7B%7D,%22complexKeywordsRestriction%22:%7B%22keyword%22:%5B%7B%22type%22:%22BROAD%22,%22value%22:%22"+collection['title'].replace(' ','+').lower()+"%22%7D%5D%7D%7D%5D,%22requestOptions%22:%7B%22property%22:%22%22,%22backend%22:%22IZG%22,%22category%22:0%7D,%22userConfig%22:%7B%22userType%22:%22USER_TYPE_LEGIT_USER%22%7D%7D&token="+tokens[collection['slug']]+'&tz='+tz
#   response = requests.get(search)
#   if response.status_code==200:
#     data=json.loads(response.text[6:])['default']['timelineData']
#     for i in data:
#       stats[collection['slug']+i['time']+'000']['trend-value']=int(i['value'][0])

In [ ]:
# project_sales_stats = pd.DataFrame.from_dict(stats,orient='index').reset_index()

### NFT Collectibles Sales

#### Rarity Calculation

In [ ]:
def calculateRares(traits_rarity):
  traits_rarity.sort()
  rarity1=traits_rarity[0]
  if len(traits_rarity)==1:
    rarity2=traits_rarity[0]
  else:
    rarity2=traits_rarity[1]
  return [rarity1,rarity2]

In [ ]:
def calculateAvgRarity(traits_rarity):
  rarity=0
  for tr in traits_rarity:
    rarity=rarity+tr
  rarity=rarity/len(traits_rarity)
  return rarity

#### Sales Data Collection

In [ ]:
ALCHEMY_API_KEYS=[
    'Opp2VA7kHvcYYbYHzDdW40cJSaE-Kz4z',
    'brKsGiRFD0RjQH67pdPxD83J1LOkz2Kd',
    # 'p4bGU_z8jpNfDOf-kMNuQg5nCzkYQw4u' 
]

In [ ]:
import random

In [ ]:
seed=2
sales=[]
for collection in collectibles[:20]:
  print(collection)
  # Beause Alchmey NFT API doesn't work well with these
#   if collection['slug'] in ['cryptopunks','nouns','flufworld','xana','hashmasks']:
#     continue
  url="https://nonfungible.com/api/salesForProject?limit=100&filter=%7B%22blockTimestamp%22:[%22"+start_date+"T00:00:00.000%2B05:30%22,%22"+end_date+"T23:59:59.999%2B05:30%22],"+(("%22nftTicker%22:[%22"+collection['token']+"%22],") if 'token' in collection.keys() else '') +"%22saleType%22:[%22secondary%22]%7D&project="+collection['projectSlug']+"&orderBy=blockTimestamp&order=DESC"
  cursor=""
  while True:
    print(url)
    response = requests.get(url)
    if response.status_code!=200:
      print('ERROR: Probllem in getting nonfungible - project nfts sales')
      print('QUERY',url)
      print('RESPONSE:',response.status_code, response.text)
      break
    results=json.loads(response.text)['sales']
    if len(results)==0:
      print('No sales mentioned here')
      break
    for i in range(math.floor(len(results)/seed)):
      result=results[i*seed]
      tokenId=result['assetId']      
      rarityUrl = "https://eth-mainnet.g.alchemy.com/nft/v2/"+ALCHEMY_API_KEYS[random.randint(0,1)]+"/computeRarity?contractAddress="+collection['address']+"&tokenId="+tokenId
      rarityResponse = requests.get(rarityUrl)
      if rarityResponse.status_code!=200:
        print('ERROR: Problem in getting alchemy - project nfts rarity data')
        print('QUERY',rarityUrl)
        print('RESPONSE:',rarityResponse.status_code, rarityResponse.text)
        continue
      rarityResults=json.loads(rarityResponse.text)
      traits_rarity=[]
      for i in range(len(rarityResults)):
        if 'prevalence' in rarityResults[i].keys():
          if rarityResults[i]['prevalence']!=None:
            traits_rarity.append(rarityResults[i]['prevalence'])
          else:
            traits_rarity.append(1)
      if len(traits_rarity)==0:
        rarity1=1
        rarity2=1
        avgRarity=1
      else:    
        rarity1,rarity2=calculateRares(traits_rarity)
        avgRarity=calculateAvgRarity(traits_rarity)
      sale={
          #   Current state
          'timestamp':result['blockTimestamp']if 'blockTimestamp' in result.keys() else None, # Time of transaction
          'usdPrice':result['usdPrice'] if 'usdPrice' in result.keys() else None, # Price of nft in USD
        #   'tokenUSDPrice':result['tokenUSDPrice'] if 'tokenUSDPrice' in result.keys() else None, # Token USD Exchange

          #   Metadata
          'tokenId':result['assetId'], # Token Id
          'avgRarity':avgRarity,
          'traits':len(traits_rarity),
          'rarity1':rarity1,
          'rarity2':rarity2,
          #   'project':result['project'], # Project slug
          'project':collection['slug'], # Project slug
          'date': collection['date'], # Include any project infos
          # 'totalSupply':collection['totalSupply'],

          # Depicts network centrality of seller
          'sellerAssetMarketCount':result['sellerAssetMarketCount'] if 'sellerAssetMarketCount' in result.keys() else None, 
          'sellerAssetReceiveCount': result['sellerAssetReceiveCount'] if 'sellerAssetReceiveCount' in result.keys() else None ,  
         
          'holdTime':result['blockTimestampChange'] if 'blockTimestampChange' in result.keys() else None, # Time between 2 sales
          #   Previous state
          'prev-timestamp':result['prevBlockTimestamp']if 'prevBlockTimestamp' in result.keys() else None, # Time of previous transaction
          'prev-usdPrice':result['prevUSDPrice'] if 'prevUSDPrice' in result.keys() else None, # Previous price of nft in USD
      
      }
      print(sale)
      sales.append(sale)

    cursor=results[-1]['_cursor']
    url="https://nonfungible.com/api/salesForProject?after="+cursor+"&limit=100&filter=%7B%22blockTimestamp%22:[%22"+start_date+"T00:00:00.000%2B05:30%22,%22"+end_date+"T23:59:59.999%2B05:30%22],"+(("%22nftTicker%22:[%22"+collection['token']+"%22],") if 'token' in collection.keys() else '') +"%22saleType%22:[%22secondary%22]%7D&project="+collection['projectSlug']+"&orderBy=blockTimestamp&order=DESC"
  
collectible_sales = pd.DataFrame(sales)
print(collectible_sales)

In [ ]:
collectible_sales

In [ ]:
collectible_sales.isnull().sum()

In [ ]:
collectible_sales=collectible_sales.dropna()

In [ ]:
collectible_sales.project.unique()

## Data Processing

In [ ]:
collectible_sales['timestamp'] = collectible_sales['timestamp'].apply(lambda x: int(datetime.timestamp(datetime.strptime(x.split('T')[0],"%Y-%m-%d")))*1000)

In [ ]:
collectible_sales['prev-timestamp'] = collectible_sales['prev-timestamp'].apply(lambda x: int(datetime.timestamp(datetime.strptime(x.split('T')[0],"%Y-%m-%d")))*1000)

In [ ]:
collectible_sales['date'] = collectible_sales['date'].apply(lambda x: int(datetime.timestamp(datetime.strptime(x.split('T')[0],"%Y-%m-%d")))*1000)

In [ ]:
collectible_sales = pd.merge(collectible_sales, project_sales_stats, on = ['timestamp','project'], how='inner')

In [ ]:
renamed_columns=addColumnPrefix(['timestamp','project-salesUSD', 'project-salesToken',
       'project-uniqueBuyers', 'project-sellers', 'project-totalTransactions',
       'project-activeOwners', 'project-totalOwners', 'project-transfers',
       'project-mints', 'project-burns', 'project-avgPriceUSD',
       'project-avgPriceToken', 'tokenUSD'], 'prev')
project_sales_stats.rename(columns = renamed_columns, inplace = True)

In [ ]:
collectible_sales = pd.merge(collectible_sales, project_sales_stats, on = ['prev-timestamp','project'], how='inner')

In [ ]:
collectible_sales = pd.merge(collectible_sales, category_sales_stats['collectible'], on = ['timestamp'], how='inner')

In [ ]:
renamed_columns=addColumnPrefix(category_sales_stats['collectible'].columns, 'prev')
category_sales_stats['collectible'].rename(columns = renamed_columns, inplace = True)

In [ ]:
collectible_sales = pd.merge(collectible_sales, category_sales_stats['collectible'], on = ['prev-timestamp'], how='inner')

In [ ]:
collectible_sales = pd.merge(collectible_sales, market_sales_stats, on = ['timestamp'], how='inner')

In [ ]:
renamed_columns=addColumnPrefix(market_sales_stats.columns, 'prev')
market_sales_stats.rename(columns = renamed_columns, inplace = True)

In [ ]:
collectible_sales = pd.merge(collectible_sales, market_sales_stats, on = ['prev-timestamp'], how='inner')

In [ ]:
collectible_sales=collectible_sales.sort_values(by=['timestamp'], ascending=True)

In [ ]:
collectible_sales['timestamp']=pd.to_datetime(collectible_sales['timestamp']*1000000)
collectible_sales['date']=pd.to_datetime(collectible_sales['date']*1000000)

In [ ]:
collectible_sales.columns

In [ ]:
collectible_sales.isnull().sum()

In [ ]:
collectible_sales=collectible_sales.dropna()

In [ ]:
collectible_sales.project.unique()

In [ ]:
collectible_sales

In [ ]:
collectible_sales.to_csv('collectible_sales.csv',index=False)

In [ ]:
files.download('collectible_sales.csv')